# Scrubbing and Cleaning Data - Lab

## Introduction

In the previous labs, you joined the data from our separate files into a single DataFrame.  In this lab, you'll scrub the data to get it ready for exploration and modeling!

## Objectives

You will be able to:

* Perform the full data cleaning process for a dataset
* Identify and deal with null values appropriately
* Remove unnecessary columns


## Getting Started

You'll find the resulting dataset from your work in the _Obtaining Data_ Lab stored within the file `'Lego_data_merged.csv'`.  

In the cells below:

* Import `pandas` and set the standard alias. 
* Import `numpy` and set the standard alias. 
* Import `matplotlib.pyplot` and set the standard alias. 
* Import `seaborn` and set the alias `sns` (this is the standard alias for seaborn). 
* Use the ipython magic command to set all matplotlib visualizations to display inline in the notebook. 
* Load the dataset stored in the `'Lego_data_merged.csv'` file into a DataFrame, `df`. 
* Inspect the head of the DataFrame to ensure everything loaded correctly. 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
# Now, load in the dataset and inspect the head to make sure everything loaded correctly
df = pd.read_csv('Lego_data_merged.csv')
df.head()

,prod_id,ages,piece_count,set_name,prod_desc,prod_long_desc,theme_name,country,list_price,num_reviews,play_star_rating,review_difficulty,star_rating,val_star_rating
0,75823,6-12,277,Bird Island Egg Heist,Catapult into action and take back the eggs fr...,Use the staircase catapult to launch Red into ...,Angry Birds™,US,$29.99,2.0,4.0,Average,4.5,4.0
1,75822,6-12,168,Piggy Plane Attack,Launch a flying attack and rescue the eggs fro...,Pilot Pig has taken off from Bird Island with ...,Angry Birds™,US,$19.99,2.0,4.0,Easy,5.0,4.0
2,75821,6-12,74,Piggy Car Escape,Chase the piggy with lightning-fast Chuck and ...,Pitch speedy bird Chuck against the Piggy Car....,Angry Birds™,US,$12.99,11.0,4.3,Easy,4.3,4.1
3,21030,12+,1032,United States Capitol Building,Explore the architecture of the United States ...,Discover the architectural secrets of the icon...,Architecture,US,$99.99,23.0,3.6,Average,4.6,4.3
4,21035,12+,744,Solomon R. Guggenheim Museum®,Recreate the Solomon R. Guggenheim Museum® wit...,Discover the architectural secrets of Frank Ll...,Architecture,US,$79.99,14.0,3.2,Challenging,4.6,4.1


## Starting our Data Cleaning

To start, you'll deal with the most obvious issue: data features with the wrong data encoding.

### Checking Data Types

In the cell below, use the appropriate method to check the data type of each column. 

In [5]:
# Your code here
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10870 entries, 0 to 10869
Data columns (total 14 columns):
prod_id              10870 non-null int64
ages                 10870 non-null object
piece_count          10870 non-null int64
set_name             10870 non-null object
prod_desc            10512 non-null object
prod_long_desc       10870 non-null object
theme_name           10870 non-null object
country              10870 non-null object
list_price           10870 non-null object
num_reviews          9449 non-null float64
play_star_rating     9321 non-null float64
review_difficulty    9104 non-null object
star_rating          9449 non-null float64
val_star_rating      9301 non-null float64
dtypes: float64(4), int64(2), object(8)
memory usage: 1.2+ MB


Now, investigate some of the unique values inside of the `list_price` column.

In [11]:
# Your code here
df['list_price'].nunique()

733

In [12]:
df['list_price'].value_counts()

$24.3878     565
$36.5878     520
$12.1878     515
$18.2878     304
$42.6878     234
            ... 
$284.58        1
$275.88        1
$32.99         1
$28.535        1
$1097.939      1
Name: list_price, Length: 733, dtype: int64

### Numerical Data Stored as Strings

A common issue to check for at this stage is numeric columns that have accidentally been encoded as strings. For example, you should notice that the `list_price` column above is currently formatted as a string and contains a proceeding '$'. Remove this and convert the remaining number to a `float` so that you can later model this value. After all, your primary task is to generate model to predict the price.

> Note: While the data spans a multitude of countries, assume for now that all prices have been standardized to USD.

In [21]:
# Your code here
# take away dollar sign
for index, val in enumerate(df['list_price']):
    df['list_price'][index] = val.replace('$', '')
df['list_price']

C:\Users\Joey\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


0          29.99
1          19.99
2          12.99
3          99.99
4          79.99
          ...   
10865    42.6878
10866    30.4878
10867    30.4878
10868    18.2878
10869    18.2878
Name: list_price, Length: 10870, dtype: object

In [24]:
df['list_price'] = df['list_price'].astype('float64')

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10870 entries, 0 to 10869
Data columns (total 14 columns):
prod_id              10870 non-null int64
ages                 10870 non-null object
piece_count          10870 non-null int64
set_name             10870 non-null object
prod_desc            10512 non-null object
prod_long_desc       10870 non-null object
theme_name           10870 non-null object
country              10870 non-null object
list_price           10870 non-null float64
num_reviews          9449 non-null float64
play_star_rating     9321 non-null float64
review_difficulty    9104 non-null object
star_rating          9449 non-null float64
val_star_rating      9301 non-null float64
dtypes: float64(5), int64(2), object(7)
memory usage: 1.2+ MB


### Detecting and Dealing With Null Values

Next, it's time to check for null values. How to deal with the null values will be determined by the columns containing them, and how many null values exist in each.  
 
In the cell below, get a count of how many null values exist in each column in the DataFrame. 

In [28]:
# Your code here
df.isna().sum() / len(df)

prod_id              0.000000
ages                 0.000000
piece_count          0.000000
set_name             0.000000
prod_desc            0.032935
prod_long_desc       0.000000
theme_name           0.000000
country              0.000000
list_price           0.000000
num_reviews          0.130727
play_star_rating     0.142502
review_difficulty    0.162466
star_rating          0.130727
val_star_rating      0.144342
dtype: float64

Now, get some descriptive statistics for each of the columns. You want to see where the minimum and maximum values lie.  

In [29]:
# Your code here
df.describe()

,prod_id,piece_count,list_price,num_reviews,play_star_rating,star_rating,val_star_rating
count,1.087000e+04,10870.000000,10870.000000,9449.000000,9321.000000,9449.000000,9301.000000
mean,6.181634e+04,503.936431,67.309137,17.813737,4.355413,4.510319,4.214439
std,1.736390e+05,831.209318,94.669414,38.166693,0.617272,0.516463,0.670906
min,6.300000e+02,1.000000,2.272400,1.000000,1.000000,1.800000,1.000000
25%,2.112300e+04,97.000000,21.899000,2.000000,4.000000,4.300000,4.000000
50%,4.207350e+04,223.000000,36.587800,6.000000,4.500000,4.600000,4.300000
75%,7.124800e+04,556.000000,73.187800,14.000000,4.800000,5.000000,4.700000
max,2.000431e+06,7541.000000,1104.870000,367.000000,5.000000,5.000000,5.000000


Now that you have a bit more of a understanding of each of these features you can make an informed decision about the best strategy for dealing with the various null values. 

Some common strategies for filling null values include:
* Using the mean of the feature
* Using the median of the feature
* Inserting a random value from a normal distribution with the mean and std of the feature
* Binning

Given that most of the features with null values concern user reviews of the lego set, it is reasonable to wonder whether there is strong correlation between these features in the first place. Before proceeding, take a minute to investigate this hypothesis.

In [35]:
# Investigate whether multicollinearity exists between the review features 
# (num_reviews, play_star_rating, star_rating, val_star_rating)
features = ['num_reviews', 'play_star_rating','star_rating','val_star_rating']
review_corr = df[features].corr()
review_corr.style.background_gradient(cmap = 'Greens')

,num_reviews,play_star_rating,star_rating,val_star_rating
num_reviews,1,-0.0608844,0.0045408,0.0266639
play_star_rating,-0.0608844,1,0.619246,0.484341
star_rating,0.0045408,0.619246,1,0.731538
val_star_rating,0.0266639,0.484341,0.731538,1


Note that there is substantial correlation between the `play_star_rating`, `star_rating` and `val_star_rating`. While this could lead to multicollinearity in your eventual regression model, it is too early to clearly determine this at this point. Remember that multicollinearity is a relationship between 3 or more variables while correlation simply investigates the relationship between two variables.

Additionally, these relationships provide an alternative method for imputing missing values: since they appear to be correlated, you could use these features to help impute missing values in the others features. For example, if you are missing the `star_rating` for a particular row but have the `val_star_rating` for that same entry, it seems reasonable to assume that it is a good estimate for the missing `star_rating` value as they are highly correlated. That said, doing so does come with risks; indeed you would be further increasing the correlation between these features which could further provoke multicollinearity in the final model.

Investigate if you could use one of the other star rating features when one is missing. How many rows have one of `play_star_rating`, `star_rating` and `val_star_rating` missing, but not all three.

In [46]:
# Your code here
# Number missing all three: 1421
# features.remove('num_reviews')
df[features].isna().sum(axis=1).value_counts(normalize=True)

0    0.855658
3    0.130727
2    0.011776
1    0.001840
dtype: float64

Well, it seems like when one is missing, the other two are also apt to be missing. While this has been a bit of an extended investigation, simply go ahead and fill the missing values with that feature's median. Fill in the missing values of `review_difficulty` feature  with string `'unknown'`.

In [52]:
# Your code here
df['review_difficulty'].fillna('unknown', inplace=True)
df['review_difficulty'].value_counts(dropna=False)

Easy                3826
Average             3357
unknown             1766
Challenging          995
Very Easy            905
Very Challenging      21
Name: review_difficulty, dtype: int64

In [57]:
for col in df.describe().columns:
    df[col].fillna(df[col].median(), inplace=True)
df.isna().sum()

prod_id                0
ages                   0
piece_count            0
set_name               0
prod_desc            358
prod_long_desc         0
theme_name             0
country                0
list_price             0
num_reviews            0
play_star_rating       0
review_difficulty      0
star_rating            0
val_star_rating        0
dtype: int64

In [58]:
df.describe()

,prod_id,piece_count,list_price,num_reviews,play_star_rating,star_rating,val_star_rating
count,1.087000e+04,10870.000000,10870.000000,10870.000000,10870.000000,10870.000000,10870.000000
mean,6.181634e+04,503.936431,67.309137,16.269365,4.376017,4.522042,4.226789
std,1.736390e+05,831.209318,94.669414,35.806576,0.573827,0.482468,0.621323
min,6.300000e+02,1.000000,2.272400,1.000000,1.000000,1.800000,1.000000
25%,2.112300e+04,97.000000,21.899000,3.000000,4.100000,4.300000,4.000000
50%,4.207350e+04,223.000000,36.587800,6.000000,4.500000,4.600000,4.300000
75%,7.124800e+04,556.000000,73.187800,12.000000,4.700000,4.900000,4.600000
max,2.000431e+06,7541.000000,1104.870000,367.000000,5.000000,5.000000,5.000000


## Normalizing the Data

Now, you'll need to convert all of our numeric columns to the same scale by **_normalizing_** our dataset.  Recall that you normalize a dataset by converting each numeric value to it's corresponding z-score for the column, which is obtained by subtracting the column's mean and then dividing by the column's standard deviation for every value. 


In the cell below:

* Normalize the numeric X features by subtracting the column mean and dividing by the column standard deviation. 
(Don't bother to normalize the `list_price` as this is the feature you will be predicting.)

In [59]:
def normalize_col(col):
    return (col - col.mean()) / col.std()

In [63]:
normalize_col(df['star_rating']).describe().round(2)

count    10870.00
mean        -0.00
std          1.00
min         -5.64
25%         -0.46
50%          0.16
75%          0.78
max          0.99
Name: star_rating, dtype: float64

In [65]:
# Your code here
df_norm = df.copy()
for cols in df_norm.describe().columns:
    df_norm[cols] = normalize_col(df_norm[cols])
df_norm.describe().round(2)

,prod_id,piece_count,list_price,num_reviews,play_star_rating,star_rating,val_star_rating
count,10870.00,10870.00,10870.00,10870.00,10870.00,10870.00,10870.00
mean,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00
std,1.00,1.00,1.00,1.00,1.00,1.00,1.00
min,-0.35,-0.61,-0.69,-0.43,-5.88,-5.64,-5.19
25%,-0.23,-0.49,-0.48,-0.37,-0.48,-0.46,-0.37
50%,-0.11,-0.34,-0.32,-0.29,0.22,0.16,0.12
75%,0.05,0.06,0.06,-0.12,0.56,0.78,0.60
max,11.16,8.47,10.96,9.80,1.09,0.99,1.24


In [67]:
df_norm

,prod_id,ages,piece_count,set_name,prod_desc,prod_long_desc,theme_name,country,list_price,num_reviews,play_star_rating,review_difficulty,star_rating,val_star_rating
0,0.080665,6-12,-0.273020,Bird Island Egg Heist,Catapult into action and take back the eggs fr...,Use the staircase catapult to launch Red into ...,Angry Birds™,US,-0.394205,-0.398512,-0.655279,Average,-0.045687,-0.365010
1,0.080660,6-12,-0.404154,Piggy Plane Attack,Launch a flying attack and rescue the eggs fro...,Pilot Pig has taken off from Bird Island with ...,Angry Birds™,US,-0.499836,-0.398512,-0.655279,Easy,0.990651,-0.365010
2,0.080654,6-12,-0.517242,Piggy Car Escape,Chase the piggy with lightning-fast Chuck and ...,Pitch speedy bird Chuck against the Piggy Car....,Angry Birds™,US,-0.573777,-0.147162,-0.132473,Easy,-0.460222,-0.204063
3,-0.234892,12+,0.635296,United States Capitol Building,Explore the architecture of the United States ...,Discover the architectural secrets of the icon...,Architecture,US,0.345210,0.187972,-1.352353,Average,0.161581,0.117830
4,-0.234863,12+,0.288812,Solomon R. Guggenheim Museum®,Recreate the Solomon R. Guggenheim Museum® wit...,Discover the architectural secrets of Frank Ll...,Architecture,US,0.133949,-0.063378,-2.049427,Challenging,0.161581,-0.204063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10865,0.052331,7-14,-0.300690,The Riddler™ Riddle Racer,Battle with Batman™ against The Riddler™ and h...,Take on four Super-Villains at once in THE LEG...,THE LEGO® BATMAN MOVIE,PT,-0.260077,0.048333,-0.306741,Easy,-0.667489,0.278777
10866,0.052371,7-14,-0.360843,Scarecrow™ Special Delivery,Help Batman™ to stop Scarecrow™ delivering a s...,Save Gotham City Energy Facility from a fear g...,THE LEGO® BATMAN MOVIE,PT,-0.388947,-0.230945,-0.481010,Average,-0.252954,-0.204063
10867,0.052319,7-14,-0.364453,Mr. Freeze™ Ice Attack,Protect the power plant from Mr. Freeze’s ice ...,Help Batman™ defend the Gotham City Energy Fac...,THE LEGO® BATMAN MOVIE,PT,-0.388947,0.076261,-0.481010,Easy,0.161581,0.278777
10868,0.052388,6-12,-0.436637,Scarecrow™ Fearful Face-off,Power into the sky to stop Scarecrow’s fear ga...,Team up with Batman™ to stop Scarecrow™ spread...,THE LEGO® BATMAN MOVIE,PT,-0.517816,-0.258873,-0.655279,Easy,-0.874757,0.278777


## Saving Your Results

While you'll once again practice one-hot encoding as you would to preprocess data before fitting a model, saving such a reperesentation of the data will eat up additional disk space. After all, a categorical variable with 10 bins will be transformed to 10 seperate features when passed through `pd.get_dummies()`. As such, while further practice is worthwhile, save your DataFrame as-is for now.

In [66]:
# Your code here
df_norm.to_csv('Lego_data_normalized.csv')

## One-Hot Encoding Categorical Columns

As a final step, you'll need to deal with the categorical columns by **_one-hot encoding_** them into binary variables via the `pd.get_dummies()` function.  

When doing this, you may also need to subset the appropriate features to avoid encoding the wrong data. The `get_dummies()` function by default converts all columns with *object* or *category* dtype. However, you should always check the result of calling `get_dummies()` to ensure that only the categorical variables have been transformed. Consult the [documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) for more details. If you are ever unsure of the data types, call the `.info()` method.

In the cell below, subset to the appropriate predictive features and then use `pd.get_dummies()` to one-hot encode the dataset properly.

In [69]:
# Your code here
df_norm_ohe = pd.get_dummies(df_norm)
df_norm_ohe

,prod_id,piece_count,list_price,num_reviews,play_star_rating,star_rating,val_star_rating,ages_10+,ages_10-14,ages_10-16,...,country_NZ,country_PL,country_PT,country_US,review_difficulty_Average,review_difficulty_Challenging,review_difficulty_Easy,review_difficulty_Very Challenging,review_difficulty_Very Easy,review_difficulty_unknown
0,0.080665,-0.273020,-0.394205,-0.398512,-0.655279,-0.045687,-0.365010,0,0,0,...,0,0,0,1,1,0,0,0,0,0
1,0.080660,-0.404154,-0.499836,-0.398512,-0.655279,0.990651,-0.365010,0,0,0,...,0,0,0,1,0,0,1,0,0,0
2,0.080654,-0.517242,-0.573777,-0.147162,-0.132473,-0.460222,-0.204063,0,0,0,...,0,0,0,1,0,0,1,0,0,0
3,-0.234892,0.635296,0.345210,0.187972,-1.352353,0.161581,0.117830,0,0,0,...,0,0,0,1,1,0,0,0,0,0
4,-0.234863,0.288812,0.133949,-0.063378,-2.049427,0.161581,-0.204063,0,0,0,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10865,0.052331,-0.300690,-0.260077,0.048333,-0.306741,-0.667489,0.278777,0,0,0,...,0,0,1,0,0,0,1,0,0,0
10866,0.052371,-0.360843,-0.388947,-0.230945,-0.481010,-0.252954,-0.204063,0,0,0,...,0,0,1,0,1,0,0,0,0,0
10867,0.052319,-0.364453,-0.388947,0.076261,-0.481010,0.161581,0.278777,0,0,0,...,0,0,1,0,0,0,1,0,0,0
10868,0.052388,-0.436637,-0.517816,-0.258873,-0.655279,-0.874757,0.278777,0,0,0,...,0,0,1,0,0,0,1,0,0,0


That's it! You've now successfully scrubbed your dataset -- you're now ready for data exploration and modeling!

## Summary

In this lesson, you gained practice with scrubbing and cleaning data. Specifically, you addressed an incorrect data type, detected and dealt with null values, checked for multicollinearity, and transformed data. Congrats on performing the full data cleaning process for a dataset!